In [1]:
import requests

def verify_doi_validity(doi_input):
    """
    Checks if a DOI exists by querying the doi.org resolver directly.
    Returns metadata if valid, or an error status if invalid.
    """
    # Clean the input to ensure we only have the DOI string (e.g., 10.xxx/yyy)
    clean_doi = doi_input.replace("https://doi.org/", "").replace("http://doi.org/", "")
    
    url = f"https://doi.org/{clean_doi}"
    
    # Header is crucial: It tells doi.org we want JSON data, not a webpage redirect.
    headers = {
        "Accept": "application/vnd.citationstyles.csl+json"
    }

    try:
        # 'allow_redirects=True' is needed because doi.org often redirects to the publisher's API
        response = requests.get(url, headers=headers, allow_redirects=True, timeout=10)
        
        if response.status_code == 200:
            try:
                data = response.json()
            except ValueError:
                return {"status": "Error", "details": "Response was not valid JSON."}
            
            # Extracting Title
            title = data.get('title', 'No Title Available')
            if isinstance(title, list) and len(title) > 0:
                title = title[0]
            
            # Extracting Journal/Container Name
            journal = data.get('container-title', 'Unknown Journal')
            if isinstance(journal, list) and len(journal) > 0:
                journal = journal[0]

            # Extracting First Author Last Name
            author_lastname = "Unknown Author"
            if 'author' in data and len(data['author']) > 0:
                author_lastname = data['author'][0].get('family', 'Unknown Author')

            return {
                "status": "Valid",
                "real_title": title,
                "journal": journal,
                "first_author": author_lastname
            }
            
        elif response.status_code == 404:
            return {"status": "Invalid", "details": "DOI does not exist in global registry."}
        else:
            return {"status": "Error", "details": f"HTTP Status Code: {response.status_code}"}

    except Exception as e:
        return {"status": "Connection Error", "details": str(e)}

# --- Usage Example ---

# List of DOIs extracted from the suspicious article
doi_list_to_check = [
    "10.1038/nature123",          # Example: A likely invalid DOI
    "10.1126/science.fake.999",   # Example: Another invalid DOI
    "10.1007/s10701-005-9016-x"   # Example: A valid DOI (Real Physics paper)
]

print(f"{'DOI String':<30} | {'Status':<10} | {'Actual Title (from Database)'}")
print("-" * 100)

for doi in doi_list_to_check:
    result = verify_doi_validity(doi)
    
    if result['status'] == "Valid":
        # Truncate title to fit the screen
        safe_title = str(result['real_title'])[:50] + "..." 
        print(f"{doi:<30} | {result['status']:<10} | {safe_title}")
    else:
        print(f"{doi:<30} | {result['status']:<10} | {result.get('details', '-')}")


DOI String                     | Status     | Actual Title (from Database)
----------------------------------------------------------------------------------------------------
10.1038/nature123              | Invalid    | DOI does not exist in global registry.
10.1126/science.fake.999       | Invalid    | DOI does not exist in global registry.
10.1007/s10701-005-9016-x      | Valid      | Physics and the Real World...


in this section 